# 1.BGGM Plot

## 1.1 Networks

In [11]:
import imageio
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import timedelta

critical_date=pd.to_datetime(['2013-01-01','2014-10-30','2015-12-17','2019-08-01','2021-11-04','2023-06-01'])

# Categories for developed and emerging countries
developed_countries = ['AUS', 'CAN', 'FRA', 'GER', 'JPN', 'SIN', 'UK', 'US']
emerging_countries = ['ARG', 'BRA', 'CHN', 'IND', 'KOR', 'MEX', 'RUS', 'THA']

# diction
country_code_to_name = {
    'AUS': 'Australia',
    'CAN': 'Canada',
    'FRA': 'France',
    'GER': 'Germany',
    'JPN': 'Japan',
    'UK': 'UK',
    'US': 'US',
    'ARG': 'Argentina',
    'BRA': 'Brazil',
    'CHN': 'China',
    'IND': 'India',
    'KOR': 'Korea',
    'MEX': 'Mexico',
    'RUS': 'Russia',
    'SIN': 'Singapore',
    'THA': 'Thailand'
}

In [ ]:
for mkt in ['composite','bond', 'forex', 'stock', 'gold']:

    image_frames = []

    path = f'DataAnalysis\\BGGM Analysis\\BGGMnetworks\\{mkt}'
    if not os.path.exists(path):
        os.mkdir(path)

    gif_path = path
    node_impact_pd=pd.DataFrame()
    #df_market=pd.read_csv('ctg_market\\'+mkt+'_Market.csv')
    df_market=pd.read_csv(f'DataAnalysis\\CTG Analysis\\summed_ctg_index\\{mkt}_Market.csv')
    df_market['date'] = pd.to_datetime(df_market['date'])


    for k in range(5):
        # Load the dataset
        df = pd.read_csv(f'Data\BGGM\pcorr_{mkt}\\pcorr_{k+1}.csv')
        df.set_index('Unnamed: 0', inplace=True)

        # Create an empty graph
        G = nx.Graph()

        # Add nodes to the graph
        for country in developed_countries:
            G.add_node(country, category='developed')
        for country in emerging_countries:
            G.add_node(country, category='emerging')

        # Add edges to the graph based on the DataFrame
        for i, row in df.iterrows():
            for j, cell in row.items():
                if i != j and cell != 0:  # Exclude self-loops and zero weights
                    color = 'green' if cell > 0 else 'orange'
                    G.add_edge(i, j, weight=abs(cell), color=color)

        # Calculate the positions
        n = len(G.nodes)
        angle = np.linspace(0, 2*np.pi, n, endpoint=False)
        sorted_nodes = developed_countries + [node for node in G.nodes if node not in developed_countries]
        pos = {}
        for idx, node in enumerate(sorted_nodes):
            pos[node] = (np.cos(angle[idx]+np.pi/n), np.sin(angle[idx]+np.pi/n))

        nodes=list(G.nodes)

        # Calculate the node impact
        node_impact = {node: 0 for node in nodes}
        node_impact

        current_start_date = critical_date[k]
        current_end_date = critical_date[k+1]
        print(f"Analyzing BGGM Network from {current_start_date.date()} to {current_end_date.date()}")

        # Add edges and attributes, update node sizes
        for node in nodes:
            df_filtered = df_market[(df_market['date'] >= pd.Timestamp(current_start_date.date())) & (df_market['date'] < pd.Timestamp(current_end_date.date()))]
            node_impact[node] = np.average(df_filtered[node].dropna())

        node_impact_pd[f'{current_start_date.date()}-{current_end_date.date()}']=pd.Series(node_impact)


        # Calculate node sizes proportional to their impact
        total_impact = sum(node_impact.values())
        #print(total_impact)
        alpha = 1000 if mkt=='composite' else 4000
        node_sizes = {node: (impact ) * alpha for node, impact in node_impact.items()}


        # Set up plot with a white background
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.set_facecolor('white')
        plt.axis('off')
        ax.set_aspect('equal', 'box')

        # Draw the graph
        node_colors = ['blue' if node in developed_countries else 'red' for node in G.nodes()]
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=[node_sizes[node] for node in G.nodes()],)  # Increased node size to 500
        edges = G.edges()
        edge_colors = [G[u][v]['color'] for u, v in edges]
        edge_weights = [G[u][v]['weight'] * 10 for u, v in edges]
        nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color=edge_colors, width=edge_weights)
        nx.draw_networkx_labels(G, pos)

        # Show the plot
        #plt.title('Modified Network of Partial Correlations between Countries')
        plt.tight_layout()
        tmp_image_path = gif_path+f"\\BGGM_Network_from_{current_start_date.date()}_to_{current_end_date.date()}.png"
        plt.savefig(tmp_image_path, facecolor='white', dpi=300)
        plt.close(fig)

        # Append the image to the list of image frames
        image_frames.append(imageio.imread(tmp_image_path))


    # Create the GIF using the list of image frames
    #imageio.mimsave(gif_path+f'\BGGM_network.gif', image_frames, duration=1)

## 1.2 Heatmaps

In [5]:
# Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import imageio
import numpy as np

for mkt in ['composite','bond', 'forex', 'stock', 'gold']:

    image_frames = []

    for i in range(1,6):

        #if not os.path.exists(f'pcorr\\tables\\pcorr_{i}.csv'):
        #    print(f"File does not exist: pcorr_{i}.csv")
        #    continue

        # Load the dataset
        df = pd.read_csv(f'Data\\BGGM\\pcorr_'+mkt+f'\\pcorr_{i}.csv')

        # Define the list of developed and developing countries based on their UN codes in the dataset
        developed_countries = ['AUS', 'CAN', 'FRA', 'GER', 'JPN', 'SIN', 'UK', 'US']
        emerging_countries = ['ARG', 'BRA', 'CHN', 'IND', 'KOR', 'MEX', 'RUS', 'THA']

        # Combine the two lists to form the new order
        new_order = developed_countries + emerging_countries

        # Reorder the DataFrame according to the new list
        df_reorder = df.set_index('Unnamed: 0').loc[new_order, new_order].reset_index()

        # Modify the DataFrame to set the diagonal values to 1
        df_reorder_modified = df_reorder.set_index('Unnamed: 0')
        for country in df_reorder_modified.index:
            df_reorder_modified.at[country, country] = 1

        # Setting up the plot style
        sns.set(style="white")

        # Create the heatmap with the reordered DataFrame
        plt.figure(figsize=(14, 12))
        heatmap = sns.heatmap(df_reorder_modified, annot=True, fmt=".2f", cmap="coolwarm", square=True, linewidths=.5, cbar_kws={"shrink": .75}, vmin=-1, vmax=1)

        # Add title and labels
        #plt.title('Partial Correlation Heatmap')
        plt.xlabel('')
        plt.ylabel('')

        # Show the plot
        plt.tight_layout()
        path = os.getcwd()+f'\\DataAnalysis\\BGGM Analysis\\BGGMheatmaps\\{mkt}'
        if not os.path.exists(path):
            os.mkdir(path)
        tmp_image_path=path+f'\\heatmap_{i}.png'
        plt.savefig(tmp_image_path,dpi=300)
        plt.close()
        #plt.show()

        # Append the image to the list of image frames
        #image_frames.append(imageio.imread(tmp_image_path))


    # Create the GIF using the list of image frames
    #imageio.mimsave(f'pcorr\heatmaps\heatmap.gif', image_frames, duration=1)


# 2. Statistical Analysis

## 2.1 Developed Countries v.s. Emerging countries

In [8]:
import pandas as pd
import numpy as np


# Define developed and emerging countries
developed_countries = ['AUS', 'CAN', 'FRA', 'GER', 'JPN', 'SIN', 'UK', 'US']
emerging_countries = ['ARG', 'BRA', 'CHN', 'IND', 'KOR', 'MEX', 'RUS', 'THA']

for mkt in ['composite','bond', 'forex', 'stock', 'gold']:

    result=[]

    for k in range(1,6):

        # Load the partial correlation matrix from the provided CSV file
        file_path = f'Data\\BGGM\\pcorr_'+mkt+f'\\pcorr_{k}.csv'
        pcorr_df = pd.read_csv(file_path, index_col=0)



        # Initialize counts for each category
        count_pos_developed, count_neg_developed, count_zero_developed = 0, 0, 0
        count_pos_mixed, count_neg_mixed, count_zero_mixed = 0, 0, 0
        count_pos_emerging, count_neg_emerging, count_zero_emerging = 0, 0, 0

        # Loop through the matrix to count positive, negative, and zero values
        for i, country1 in enumerate(pcorr_df.index):
            for j, country2 in enumerate(pcorr_df.columns):
                if i >= j:
                    continue
                value = pcorr_df.at[country1, country2]

                # Developed-developed
                if country1 in developed_countries and country2 in developed_countries:
                    count_pos_developed += (value > 0)
                    count_neg_developed += (value < 0)
                    count_zero_developed += (value == 0)

                # Developed-emerging
                elif (country1 in developed_countries and country2 in emerging_countries) or \
                     (country1 in emerging_countries and country2 in developed_countries):
                    count_pos_mixed += (value > 0)
                    count_neg_mixed += (value < 0)
                    count_zero_mixed += (value == 0)

                # Emerging-emerging
                elif country1 in emerging_countries and country2 in emerging_countries:
                    count_pos_emerging += (value > 0)
                    count_neg_emerging += (value < 0)
                    count_zero_emerging += (value == 0)

        # Calculate proportions
        total_developed = count_pos_developed + count_neg_developed + count_zero_developed
        total_mixed = count_pos_mixed + count_neg_mixed + count_zero_mixed
        total_emerging = count_pos_emerging + count_neg_emerging + count_zero_emerging

        prop_pos_developed = count_pos_developed / total_developed
        prop_neg_developed = count_neg_developed / total_developed
        prop_zero_developed = count_zero_developed / total_developed

        prop_pos_mixed = count_pos_mixed / total_mixed
        prop_neg_mixed = count_neg_mixed / total_mixed
        prop_zero_mixed = count_zero_mixed / total_mixed

        prop_pos_emerging = count_pos_emerging / total_emerging
        prop_neg_emerging = count_neg_emerging / total_emerging
        prop_zero_emerging = count_zero_emerging / total_emerging

        # Create a results dictionary
        results_dict = {
            'Phase': f'{k}',
            'DD_Positive': prop_pos_developed, 'DD_Negative': prop_neg_developed, 'DD_Zero': prop_zero_developed,
            'DE_Positive': prop_pos_mixed, 'DE_Negative': prop_neg_mixed, 'DE_Zero': prop_zero_mixed,
            'EE_Positive': prop_pos_emerging, 'EE_Negative': prop_neg_emerging, 'EE_Zero': prop_zero_emerging,
        }

        result.append(results_dict)

    result_pd=pd.DataFrame(result).set_index('Phase')
    result_pd.to_csv(f'DataAnalysis\\BGGM Analysis\\StatisticalAnalysis\\developed_emerging_analysis\\{mkt}.csv')

## 2.2 Measurement

### CDC & CDI

In [3]:
import pandas as pd
import networkx as nx

# Define your critical dates
critical_date = ['2013-01-01', '2014-10-30', '2015-12-17', '2019-08-01', '2021-11-04', '2023-06-01']

# Initialize DataFrames to store the results
average_degrees = pd.DataFrame(index=[f'phase {i}' for i in range(1, len(critical_date))], columns=['composite','bond', 'forex', 'stock', 'gold'])
average_weights = pd.DataFrame(index=[f'phase {i}' for i in range(1, len(critical_date))], columns=['composite','bond', 'forex', 'stock', 'gold'])

# Loop over each market and phase
for mkt in ['composite','bond', 'forex', 'stock', 'gold']:
    for i in range(1, len(critical_date)):
        # Step 1: Load the CSV Data into a DataFrame
        try:
            df_pcorr = pd.read_csv(f'Data\\BGGM\\pcorr_'+mkt+f'\\pcorr_{i}.csv', index_col=0)

            # Step 2: Create a Graph
            G = nx.Graph()

            # Step 3: Add Edges to the Graph
            for idx, row in df_pcorr.iterrows():
                for col, cell in row.items():
                    if idx != col and cell != 0:  # Exclude self-loops and zero values
                        G.add_edge(idx, col, weight=abs(cell))

            # Step 4: Calculate the Average Degree
            average_degree = sum(dict(G.degree()).values()) / 16 #G.number_of_nodes()

            # Step 5: Calculate the Average Weight of Non-Zero Edges
            edge_weights = [data['weight'] for _, _, data in G.edges(data=True)]
            average_edge_weight = sum(edge_weights) / len(edge_weights) if edge_weights else 0

            # Store the results in the respective DataFrames
            average_degrees.at[f'phase {i}', mkt] = average_degree
            average_weights.at[f'phase {i}', mkt] = average_edge_weight
        except FileNotFoundError:
            print(f"File for market '{mkt}' and phase '{i}' not found.")

# Output the DataFrames
average_degrees.to_csv('DataAnalysis\BGGM Analysis\StatisticalAnalysis\Measurements\closeness.csv')
average_weights.to_csv('DataAnalysis\BGGM Analysis\StatisticalAnalysis\Measurements\intensity.csv')


### SSP & SST

In [1]:
import pandas as pd
import numpy as np

critical_date = pd.to_datetime(['2013-01-01', '2014-10-30', '2015-12-17', '2019-08-01', '2021-11-04', '2023-06-01'])


# Loop over each market and phase
for mkt in ['composite','bond', 'forex', 'stock', 'gold']:
    # Load the market data
    df_market = pd.read_csv('DataAnalysis\\CTG Analysis\\summed_ctg_index\\'+mkt+'_Market.csv')
    df_market['date'] = pd.to_datetime(df_market['date'])

    # Define nodes and critical dates
    nodes = df_market.columns[1:].tolist()


    # Initialize an empty DataFrame to store node impact
    node_impact_pd = pd.DataFrame()

    # Calculate node impact for each phase
    for k in range(5):
        node_impact = {node: 0 for node in nodes}
        current_start_date = critical_date[k]
        current_end_date = critical_date[k + 1]
        for node in nodes:
            df_filtered = df_market[(df_market['date'] >= current_start_date) & (df_market['date'] < current_end_date)]
            node_impact[node] = np.average(df_filtered[node].dropna())
        node_impact_pd[f'phase{k + 1}'] = pd.Series(node_impact)

    # Load the partial correlation data for each phase
    #pcorr_files = [f'pcorr_{i + 1}.csv' for i in range(5)]
    #pcorr_dfs = [pd.read_csv(file).set_index('Unnamed: 0') for file in pcorr_files]

    # Initialize DataFrames to store Spillover and Stability Measurements
    spillover_df = pd.DataFrame(index=nodes)
    stability_df = pd.DataFrame(index=nodes)

    # Calculate Spillover and Stability Measurements for each phase with corrected total node impact
    for k in range(5):

        pcorr_df=pd.read_csv(f'Data\BGGM\pcorr_{mkt}\\pcorr_{k+1}.csv').set_index('Unnamed: 0')

        spillover_values = {}
        stability_values = {}
        node_impact_phase = node_impact_pd[f'phase{k + 1}']
        total_node_impact_current_phase = node_impact_phase.sum()
        for node in nodes:
            pos_corr = pcorr_df.loc[node][pcorr_df.loc[node] > 0]
            neg_corr = pcorr_df.loc[node][pcorr_df.loc[node] < 0]
            spillover = (pos_corr * (node_impact_phase[node] + node_impact_phase[pos_corr.index])).sum() / total_node_impact_current_phase
            stability = (abs(neg_corr) * (node_impact_phase[node] + node_impact_phase[neg_corr.index])).sum() / total_node_impact_current_phase
            spillover_values[node] = spillover
            stability_values[node] = stability
        spillover_df[f'phase{k + 1}'] = pd.Series(spillover_values)
        stability_df[f'phase{k + 1}'] = pd.Series(stability_values)

    # Save the DataFrames to CSV files
    spillover_df.to_csv(f'DataAnalysis\\BGGM Analysis\\StatisticalAnalysis\\Measurements\\Spillover_Measurements_{mkt}.csv')
    stability_df.to_csv(f'DataAnalysis\\BGGM Analysis\\StatisticalAnalysis\\Measurements\\Stability_Measurements_{mkt}.csv')